In [ ]:
!pip install pymongo

In [ ]:
from pymongo import MongoClient
import gridfs
import cv2
import numpy as np
import os
from glob import glob
import urllib

In [ ]:
#MongoDB 연동
mongo_uri = "mongodb://" + urllib.parse.quote_plus("아이디") + ":" + urllib.parse.quote_plus("비밀번호") + "@ip:27017/"
client = MongoClient(mongo_uri)
db = client['imagedb']
fs = gridfs.GridFS(db)

In [ ]:
#차종,브랜드 리스트 정의
kind_list = ['hatchback', 'sedan','sportscar','suv']
maker_list = ['audi', 'bentley','benz','bmw','chevrolet','chrysler','ferrari','ford', 'hyundai', 'kia', 'lamborghini', 'landrover', 'mini', 'nissan', 'porsche', 'ssangyong', 'toyota', 'volkswagen']

In [ ]:
#MongoDB에 데이터 insert
for i in range(4):
    Collection = db[kind_list[i]]

    for j in [0,1,2,3,5,6,8,9,14,17]:
        img_list = glob('/저장경로/'+str(i)+'/'+str(j)+'/*.png')

        cnt = 0

        for img_file in img_list:

            # 이미지 불러와 RGB로
            image = cv2.imread(img_file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # ndarray를 string으로 변환
            imageString = image.tostring()

            # 이미지 저장
            imageID = fs.put(imageString, encoding='utf-8')

            maker = maker_list[j]
            name = maker + str(cnt)
            cnt += 1

            # 이미지 메타데이터 생성
            meta = {
                'name': name,
                'maker' : maker,
                'images': [
                    {
                        'imageID': imageID,
                        'shape': image.shape,
                        'dtype': str(image.dtype)
                    }
                ]
            }

            # 메타 데이터 insert
            Collection.insert_one(meta)

In [ ]:
#mongoDB 연동하여 이미지 호출
# 이미지 메타데이터 호출
image = collection.find_one({'maker': 'benz'})['images'][0]

# gridfs로 이미지 get
gOut = fs.get(image['imageID'])

# ndarray로 변환
img = np.frombuffer(gOut.read(), dtype=np.uint8)

# 이미지 사이즈 복구
img = np.reshape(img, image['shape'])

# BGR을 RGB로
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)